In [38]:
# %%
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor
import lightgbm as lgb
import json

import re
import subprocess
import time

# ----------------------------------------
# LOAD THE TRAINED MODEL
# ----------------------------------------
booster = lgb.Booster(model_file="models/final_model.lgb")

with open(f'metadata/final_metadata.json') as f:
    data = json.load(f)

description = data['description']
cols_to_take = data["cols_to_take"]

df_test = pd.read_parquet('data/df_rank_v2.parquet')

# Build test matrix
X_test = df_test[cols_to_take]

categorical_features = ["aircraft_type", "phase"]
cat_feats_actual = [c for c in categorical_features if c in X_test.columns]
for c in categorical_features:
    X_test[c] = X_test[c].astype("category")

# ----------------------------------------
# PREDICT
# ----------------------------------------
ff_kgs_pred = booster.predict(X_test)

df_test["ff_kgs"] = ff_kgs_pred
df_test["fuel_kg"] = ff_kgs_pred * df_test["seg_duration"]

/var/folders/gt/7bzbbv711312cfd_13h4fkm5z8xgxd/T/ipykernel_12428/2699182908.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[c] = X_test[c].astype("category")
/var/folders/gt/7bzbbv711312cfd_13h4fkm5z8xgxd/T/ipykernel_12428/2699182908.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[c] = X_test[c].astype("category")


In [39]:
# -------------------------------------------------
# Save submission
# -------------------------------------------------
version = 245

df_final = pd.read_parquet('data/fuel_rank_submission.parquet')
df_final["fuel_kg"] = df_test["fuel_kg"].values
df_final.to_parquet(f'data/resourceful-quiver_v{version}.parquet', index=False)

# -------------------------------------------------
# Upload to MinIO
# -------------------------------------------------
cmd_upload = [
    "mc", "cp",
    f"data/resourceful-quiver_v{version}.parquet",
    "opensky/prc-2025-resourceful-quiver"
]

subprocess.run(cmd_upload, check=True)
print(f"Uploaded resourceful-quiver_final.parquet to opensky/prc-2025-resourceful-quiver/")

cmd = [ "mc", "cat", f"opensky/prc-2025-resourceful-quiver/resourceful-quiver_v{version}.parquet_result.json" ]

while True:
    try:
        print("Trying to fetch result JSON...")
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        json_text = result.stdout
        print("Success! File found.")
        break

    except subprocess.CalledProcessError:
        print("File not ready yet. Retrying in 15 seconds...")
        time.sleep(15)
        
# Parse JSON
info = json.loads(json_text)

# Print score
print("Status:", info.get("status"))
print("File:", info.get("file"))
print("Score:", info.get("score"))
print("Used pairs:", info.get("used_pairs"))

`/Users/mfrahman/Python/PRC-2025/data/resourceful-quiver_v245.parquet` -> `opensky/prc-2025-resourceful-quiver/resourceful-quiver_v245.parquet`
Total: 816.71 KiB, Transferred: 816.71 KiB, Speed: 1016.04 KiB/s
Uploaded resourceful-quiver_final.parquet to opensky/prc-2025-resourceful-quiver/
Trying to fetch result JSON...
File not ready yet. Retrying in 15 seconds...
Trying to fetch result JSON...
Success! File found.
Status: Succeeded
File: resourceful-quiver_v245.parquet
Score: 199.9065
Used pairs: 24289
